# Spatial linear regression models

https://pysal.org/spreg/notebooks/GM_Lag_example.html

conda install -c conda-forge watermark

In [1]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00


In [2]:
#  There is an external add on for jupyter notebook that will "watermark" the notebook, giving a lot of information
# about the python version used

%load_ext watermark
%watermark

Last updated: 2024-03-12T22:50:05.362014+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [4]:
!pip install pysal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.8/331.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# we can watermark the imports as well.
# think about how much easier this is going to make detecting installation differences....

# spreg is part of pysal,  it is a spatial econometrics package

import numpy
import libpysal
import spreg
from IPython.display import HTML


%watermark -w
%watermark -iv

Watermark: 2.4.3

spreg   : 1.4.2
numpy   : 1.25.2
libpysal: 4.10



## Baltimore Housing Data

We are working here with a set of data files on Baltimore housing data
it is an ARC-GIS style data set, with a lot of different files

dbf is the attributes

In [7]:
libpysal.examples.explain("baltim")

baltim

Baltimore house sales prices and hedonics, 1978. 
----------------------------------------------------------------

* baltim.dbf: attribute data. (k=17)
* baltim.shp: Point shapefile. (n=211)
* baltim.shx: spatial index.
* baltim.tri.k12.kwt: kernel weights using a triangular kernel with 12 nearest neighbors in KWT format.
* baltim_k4.gwt: nearest neighbor weights (4nn) in GWT format.
* baltim_q.gal: queen contiguity weights in GAL format.
* baltimore.geojson: spatial weights in geojson format.

Source: Dubin, Robin A. (1992). Spatial autocorrelation and neighborhood quality. Regional Science and Urban Economics 22(3), 433-452.


The Baltimore data, in the .dbf file is read in, using a libpysal utility into a "db" variable

From the db variable the price variable is extracted and placed in y,   predictors ("exogenous variables") are extracted and
stored in a numpy array

In [8]:
# Read Baltimore data
db = libpysal.io.open(libpysal.examples.get_path("baltim.dbf"), "r")
ds_name = "baltim.dbf"

# Read dependent variable
y_name = "PRICE"
y = numpy.array(db.by_col(y_name)).T
y = y[:, numpy.newaxis]

# Read exogenous variables
x_names = ["NROOM", "NBATH", "PATIO", "FIREPL", "AC", "GAR", "AGE", "LOTSZ", "SQFT"]
x = numpy.array([db.by_col(var) for var in x_names]).T

In [9]:
print(y.shape)
print(x.shape)

(211, 1)
(211, 9)


In [10]:
# what is the db variable
type(db)

libpysal.io.iohandlers.pyDbfIO.DBF

In [11]:
dir(db)

['FORMATS',
 'MODES',
 '_By_Col',
 '_By_Row',
 '_FileIO__getIds',
 '_FileIO__ids',
 '_FileIO__rIds',
 '_FileIO__read',
 '_FileIO__registry',
 '_FileIO__setIds',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cast',
 '_col_index',
 '_complain_ifclosed',
 '_firstWrite',
 '_get_col',
 '_read',
 '_register',
 '_spec',
 '_writeHeader',
 'by_col',
 'by_col_array',
 'by_row',
 'cast',
 'check',
 'close',
 'closed',
 'dataObj',
 'dataPath',
 'f',
 'field_info',
 'field_spec',
 'flush',
 'get',
 'getType',
 'header',
 'header_size',
 'ids',
 'mode',
 'n_fields',
 'n_records',
 'open',
 'pos',
 'rIds',
 'read',
 'read_reco

In [12]:
db.field_info

[('DeletionFlag', 'C', 1, 0),
 ('STATION', 'N', 6, 0),
 ('PRICE', 'N', 10, 6),
 ('NROOM', 'N', 9, 6),
 ('DWELL', 'N', 8, 6),
 ('NBATH', 'N', 8, 6),
 ('PATIO', 'N', 8, 6),
 ('FIREPL', 'N', 8, 6),
 ('AC', 'N', 8, 6),
 ('BMENT', 'N', 8, 6),
 ('NSTOR', 'N', 8, 6),
 ('GAR', 'N', 8, 6),
 ('AGE', 'N', 10, 6),
 ('CITCOU', 'N', 8, 6),
 ('LOTSZ', 'N', 10, 6),
 ('SQFT', 'N', 9, 6),
 ('X', 'N', 10, 6),
 ('Y', 'N', 10, 6)]

In [13]:
# Read spatial data
# looking at the data above, it is pulling the queen contiguency weights

ww = libpysal.io.open(libpysal.examples.get_path("baltim_q.gal"))
w = ww.read()
ww.close()
w_name = "baltim_q.gal"
w.transform = "r"

In [14]:
type(w)

libpysal.weights.weights.W

In [15]:
w.weights

{'1': [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 '2': [0.2, 0.2, 0.2, 0.2, 0.2],
 '3': [0.5, 0.5],
 '4': [0.2, 0.2, 0.2, 0.2, 0.2],
 '5': [0.2, 0.2, 0.2, 0.2, 0.2],
 '6': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
 '7': [0.2, 0.2, 0.2, 0.2, 0.2],
 '8': [0.25, 0.25, 0.25, 0.25],
 '9': [0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666],
 '10': [0.25, 0.25, 0.25, 0.25],
 '11': [0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666],
 '12': [0.2, 0.2, 0.2, 0.2, 0.2],
 '13': [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 '14': [0.16666666666666666,
  0.16666666666666666,
  0.166

# Skip down in the notebook to the OLS method,  it is easier to understand!

# the model used here is called basic 2SLS

y = rho*W(y) +mX + epsilon

This is a spatial autoregressive model, in which the price (y) is thought to depend on the lag weights (the queen lags) and
a number of other variables in X,  plus an error term

Adding the spatial predictor term W(y) into the model is just assuming that price depends strongly on the price of neighboring
regions

The variance/covariance matrix here is hard to see,   it is too large, but it gives us the covariance of the predictors
with the spatial weight terms

The 2SLS model is a two step, partial least squares method- this is a relatively complex model, as it makes use of an SVD style decomposition of the predictors X,   so it is a specialized form of regression

I found the 2SLS based model first,  there is an ordinary least square version lower in the notebook

In [16]:
model = spreg.GM_Lag(
    y,
    x,
    w=w,
    name_y=y_name,
    name_x=x_names,
    name_w="baltim_q",
    name_ds="baltim",
    spat_diag=True,
    vm=True
)

print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      baltim
Weights matrix      :    baltim_q
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          11
S.D. dependent var  :     23.6061                Degrees of Freedom    :         200
Pseudo R-squared    :      0.7278
Spatial Pseudo R-squared:  0.6928

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -2.57627         5.52104        -0.46663         0.64077
               NROOM         0.94407         1.06097         0.88982         0.37356
               NBATH         5.59813      

In [17]:
y_pred=model.predy
print( sum((y_pred-y)**2))

[31872.62482845]


In [18]:
# using second order spatial lags for the instruments, set w_lags = 2
model2 = spreg.GM_Lag(
    y,
    x,
    w=w,
    w_lags=2,                           # using 2nd order lags, ie lags of other variables
    name_y=y_name,
    name_x=x_names,
    name_w="baltim_q",
    name_ds="baltim",
    spat_diag=True
)
print(model2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      baltim
Weights matrix      :    baltim_q
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          11
S.D. dependent var  :     23.6061                Degrees of Freedom    :         200
Pseudo R-squared    :      0.7276
Spatial Pseudo R-squared:  0.6915

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -2.88676         5.45633        -0.52907         0.59676
               NROOM         0.95274         1.06131         0.89771         0.36934
               NBATH         5.59753      

In [19]:
y_pred=model2.predy
print( sum((y_pred-y)**2))

[31909.25173849]


## Ordinary Least squares regression


the model below is on ordinary least squares linear regression model,

Y= mX +b + error

After the model is fitted,  we will check to see if there is a correlation of the erros withe the spatial weights,
which would indicate we should use a spatial weight model

In [20]:
ols2 = spreg.OLS(y, x, w, spat_diag=True,moran=True, name_y=y_name, name_x=x_names, name_ds='Baltimore_OLS', white_test=True)
print(ols2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :Baltimore_OLS
Weights matrix      :     unknown
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:     40960.5                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

-----------------------------------------------------------

In [21]:
y_pred=ols2.predy
print( sum((y_pred-y)**2))

[40960.46328547]
